# Rosbag Parser
02/11/2022
Gregory Burgess

In folder where rosbags are stored, need another folder called '/path_to_rosbags/rosbag_output/'

---
## Import Libraries

In [19]:
import bagpy
import pandas as pd
import seaborn as sea
import matplotlib.pyplot as plt
import numpy as np

from bagpy import bagreader

---
## Import Rosbags
### Insert filepath to Rosbags


In [20]:
filepath = '/home/glider-sim/dvl-nav/data/field_data/buzz_bay/jul8/rosbags/'
# filepath = '/home/gburgess/dvl-nav/data/field_data/puerto_rico/mar8/rosbags/'
# Grab all files ending with .bag in filepath
bags_raw = [bag for bag in os.listdir(path=filepath) if bag.endswith('.bag')]
# Sort rosbag files in chronological order
bags_raw.sort()

## Select Which Topics to Sort out

In [21]:
# Can look at an individual bag
b=bagreader(filepath+bags_raw[15])
b.topic_table

[INFO]  Successfully created the data folder /home/glider-sim/dvl-nav/data/field_data/buzz_bay/jul8/rosbags/2022-07-08-15-42-36_10.


,Topics,Types,Message Count,Frequency
0,/battery_pack/data,sentinel_battery_logger_msgs/BatteryPackState,117,1.866977
1,/devices/dvl/dvl,ds_sensor_msgs/Dvl,260,0.991413
2,/devices/dvl/instrument/raw,ds_core_msgs/RawData,260,0.991999
3,/devices/dvl/pd0,ds_sensor_msgs/RdiPD0,260,0.991166
4,/devices/dvl/ranges,ds_sensor_msgs/Ranges3D,260,0.991378
5,/devices/dvl/status,ds_core_msgs/Status,17,0.066667
6,/devices/spartonm2/ahrs,ds_sensor_msgs/Ahrs,2605,9.999247
7,/devices/spartonm2/instrument/raw,ds_core_msgs/RawData,2605,9.999330
8,/devices/spartonm2/spartonm2_orientation,visualization_msgs/Marker,2605,9.999223
9,/devices/spartonm2/status,ds_core_msgs/Status,18,0.066667


In [22]:
topics = [
    '/devices/spartonm2/ahrs',
    '/devices/dvl/instrument/raw',
    '/devices/dvl/pd0',
    '/devices/dvl/ranges',
    '/devices/dvl/dvl',
    '/glider/extctl/sensors/m_depth',
    '/glider/extctl/sensors/m_altitude',
    '/glider/extctl/sensors/m_altimeter_status',
    '/glider/extctl/sensors/m_dist_to_wpt',
    '/glider/extctl/sensors/m_gps_lat',
    '/glider/extctl/sensors/m_gps_lon',
    '/glider/extctl/sensors/m_lat',
    '/glider/extctl/sensors/m_lon',
    '/glider/extctl/sensors/m_pitch',
    '/glider/extctl/sensors/m_present_time',
    '/glider/extctl/sensors/m_roll',
    '/glider/extctl/sensors/m_water_depth',
    '/glider/extctl/sensors/m_heading',
    '/glider/extctl/sensors/c_wpt_lat',
    '/glider/extctl/sensors/c_wpt_lon',
    '/tritech_micron_beam',
    '/tritech_micron_data',
    '/ctd/data',
    '/battery_pack/data'
]

### Parse Bags

In [23]:
# Initialize Dictionary to hold filepaths to generated CSVs
csv_dic = {}
for t in topics:
    csv_dic[t] = []

# Parse out Data by topic and generate set of names
for bag in bags_raw:
    b=bagreader(filepath+bag)
    for t in topics:
        data = b.message_by_topic(t)
        if type(data) == str:
            csv_dic[t].append(data)

[INFO]  Successfully created the data folder /home/glider-sim/dvl-nav/data/field_data/buzz_bay/jul8/rosbags/2022-07-08-11-19-34_0.
No data on the topic:/glider/extctl/sensors/m_depth
No data on the topic:/glider/extctl/sensors/m_altitude
No data on the topic:/glider/extctl/sensors/m_altimeter_status
No data on the topic:/glider/extctl/sensors/m_dist_to_wpt
No data on the topic:/glider/extctl/sensors/m_gps_lat
No data on the topic:/glider/extctl/sensors/m_gps_lon
No data on the topic:/glider/extctl/sensors/m_lat
No data on the topic:/glider/extctl/sensors/m_lon
No data on the topic:/glider/extctl/sensors/m_pitch
No data on the topic:/glider/extctl/sensors/m_present_time
No data on the topic:/glider/extctl/sensors/m_roll
No data on the topic:/glider/extctl/sensors/m_water_depth
No data on the topic:/glider/extctl/sensors/m_heading
No data on the topic:/glider/extctl/sensors/c_wpt_lat
No data on the topic:/glider/extctl/sensors/c_wpt_lon
No data on the topic:/ctd/data
[INFO]  Successfully

In [24]:
names = []
for key in csv_dic:
    print(key)
    names.append(key.replace('/','-')[1:])


/devices/spartonm2/ahrs
/devices/dvl/instrument/raw
/devices/dvl/pd0
/devices/dvl/ranges
/devices/dvl/dvl
/glider/extctl/sensors/m_depth
/glider/extctl/sensors/m_altitude
/glider/extctl/sensors/m_altimeter_status
/glider/extctl/sensors/m_dist_to_wpt
/glider/extctl/sensors/m_gps_lat
/glider/extctl/sensors/m_gps_lon
/glider/extctl/sensors/m_lat
/glider/extctl/sensors/m_lon
/glider/extctl/sensors/m_pitch
/glider/extctl/sensors/m_present_time
/glider/extctl/sensors/m_roll
/glider/extctl/sensors/m_water_depth
/glider/extctl/sensors/m_heading
/glider/extctl/sensors/c_wpt_lat
/glider/extctl/sensors/c_wpt_lon
/tritech_micron_beam
/tritech_micron_data
/ctd/data
/battery_pack/data


In [25]:
# Concatenate all csvs into one dataframe per topic
final_dic = {}
n_count=0
for key in csv_dic:
    count = 0
    temp_dic = {}
    for csv in csv_dic[key]:
        temp_dic[key+str(count)] = (pd.read_csv(csv, on_bad_lines='skip'))
        count += 1
    temp_df = pd.concat(temp_dic)
    new_df = temp_df.reset_index()
    new_df = new_df.iloc[:, 2:]
    final_dic[names[n_count]] = new_df
    n_count += 1

In [26]:
# Save final CSVs to output folder
for key in final_dic:
    final_dic[key].to_csv(filepath+'rosbag_output/'+str(key)+'.csv')